# Testing Model 1

In [ ]:
from ultralytics import YOLO
import cv2
import supervision as sv
import numpy as np
import time
import os
from IPython.display import display


#config

model_path = "./Model1_local/Model1_local.pt" # Local trained model
test_vid_path = "../Dataset/Video"  # Path to the test video folder
output_path = "../Results/Model1_local" # Path to save annotated videos

## load model

In [2]:
model = YOLO(model_path)


## Test

In [3]:
for i,vid in enumerate(os.listdir(test_vid_path)):
    curr_vid_path = os.path.join(test_vid_path, vid)
    output_video_path = os.path.join(output_path, f"test_vid_{i}.mp4")

    print(f"Processing video: {vid}")

    cap = cv2.VideoCapture(curr_vid_path)
    tracker = sv.ByteTrack()
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    if fps <= 1 or fps > 240:  # sanity check
        print(f"⚠️ FPS read as {fps}. Defaulting to 30 FPS.")
        fps = 30
    fps = int(fps)
    
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    object_speeds = {}
    object_last_positions = {}
    object_last_times = {}

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, verbose = False)[0]
        detections = sv.Detections.from_ultralytics(results)

        # Update tracker
        detections = tracker.update_with_detections(detections)

        current_time = time.time()

        for i, (xyxy, track_id) in enumerate(zip(detections.xyxy, detections.tracker_id)):
            if track_id is None:
                continue

            x1, y1, x2, y2 = map(int, xyxy)
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            # Draw box
            label = f"ID {track_id}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
            out.write(frame)




    cap.release()
    out.release()
    cv2.destroyAllWindows()


Processing video: test_vid_4.mp4
Processing video: test_vid_5.mp4
Processing video: test_vid_7.mp4
Processing video: CVData-Foodproduct.mov
Processing video: test_vid_6.mp4
Processing video: test_vid_2.mp4
Processing video: .DS_Store
⚠️ FPS read as 0. Defaulting to 30 FPS.
Processing video: test_vid_3.mp4


OpenCV: Couldn't read video stream from file "Dataset/Video/.DS_Store"


Processing video: test_vid_1.mp4
Processing video: CvData-Boxes.mov
Processing video: 3ply Surgical Mask Making & Packing Machine (Overhead Shot).mp4
Processing video: test_vid_8.mp4
